In [1]:
import os

import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt

import cmcrameri.cm as cmc
import math

In [2]:
def myround(x, base=1):
    return base * round(x/base)

In [3]:
combined = pd.read_csv('/home/earthquakes1/homes/Rebecca/phd/stf/data/combined.csv')

In [4]:
# looks for time value of root
def f3(end_time, total_moment, time_opt, momentrate_opt, start, points_before_zero, proportion = 0.1):

    dx = time_opt[1]-time_opt[0]
    end_window = (end_time/dx)+points_before_zero
    end = int(np.floor(end_window))
    #print(start, end)
    if start == end:
        end += 1
    short = scipy.integrate.simpson(momentrate_opt[start:end], dx = dx)
    return short-(total_moment*proportion)

In [5]:
proportions_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

all_durations = []
all_root_times = []
all_moments = []
all_relative_root_times = []

results_db = pd.DataFrame(columns = ['scardec_name', 'magnitude', 'int_magnitude', 'year', 'month', 'day', '10', '20', '30', '40', '50', '60', '70', '80', '90'])

colors = {'10': 'dodgerblue',
          '20': 'darkorange',
          '30': 'green',
          '40': 'red',
          '50': 'purple',
          '60': 'blue',
          '70': 'orange',
          '80': 'green',
          '90': 'red'}


In [6]:
combined = pd.read_csv('/home/earthquakes1/homes/Rebecca/phd/stf/data/combined_scardec_ye_usgs_sigloch_isc_mag.csv')

In [7]:
combined
combined.columns = ['events', 'scardec', 'ye', 'isc', 'sigloch', 'usgs', 'mag']

In [8]:
catalog = pd.read_csv('/home/earthquakes1/homes/Rebecca/phd/stf/data/combined_m55_catalog.csv', sep = '|')

In [9]:
cols = catalog.columns
column_names = []
for c in cols:
    column_names.append(c.strip().rstrip().lower())
column_names[0] = 'catalog_id'
catalog.columns = column_names

In [10]:
catalog['year'] = catalog.apply(lambda x: x['time'][:4], axis = 1)
catalog['month'] = catalog.apply(lambda x: x['time'][5:7], axis = 1)
catalog['day'] = catalog.apply(lambda x: x['time'][8:10], axis = 1)
catalog['hour'] = catalog.apply(lambda x: x['time'][11:13], axis = 1)
catalog['minute'] = catalog.apply(lambda x: x['time'][14:16], axis = 1)

catalog['event'] = catalog.apply(lambda x: x['year'] + x['month'] + x['day'] + '_' + x['hour'] + x['minute'], axis = 1)

catalog['int_magnitude'] = catalog.apply(lambda x: myround(x['magnitude']), axis = 1)

In [11]:
catalog.drop(columns = ['contributor', 'contributorid', 'magauthor', 'eventlocationname', 'author', 'catalog', 'time'], inplace = True)

In [12]:
catalog = catalog[['event', 'catalog_id', 'year', 'month', 'day', 'hour', 'minute', 'latitude', 'longitude', 'depth/km', 'magnitude', 'int_magnitude', 'magtype']]

In [13]:
results_db

,scardec_name,magnitude,int_magnitude,year,month,day,10,20,30,40,50,60,70,80,90


In [14]:
results_db = pd.read_csv(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/all_relative_root_times.csv')
results_db.columns = ['event', '10', '20', '30', '40', '50', '60', '70', '80', '90']
results_db.sort_values(by = 'event', inplace = True, ignore_index=True)

results_10 = pd.DataFrame(columns = ['event', '0_10', '10_20', '20_30', '30_40', '40_50', '50_60', '60_70', '70_80', '80_90', '90_100'])


results_10['event'] = results_db.apply(lambda x: x['event'].split('_')[0]+'_'+x['event'].split('_')[1], axis = 1)
results_10['dataset'] = results_db.apply(lambda x: x['event'].split('_')[2], axis = 1)

results_10['0_10'] = results_db['10']
results_10['10_20'] = results_db['20'] - results_db['10']
results_10['20_30'] = results_db['30'] - results_db['20']
results_10['30_40'] = results_db['40'] - results_db['30']
results_10['40_50'] = results_db['50'] - results_db['40']
results_10['50_60'] = results_db['60'] - results_db['50']
results_10['60_70'] = results_db['70'] - results_db['60']
results_10['70_80'] = results_db['80'] - results_db['70']
results_10['80_90'] = results_db['90'] - results_db['80']
results_10['90_100'] = 1 - results_db['90']

results_10_and_catalog = pd.merge(results_10, catalog, on = 'event', how = 'inner')
results_10_and_catalog = results_10_and_catalog[['event', 'dataset', 'catalog_id', 'year', 'month', 'day', 'hour', 'minute', 'latitude', 'longitude', 'depth/km', 'magnitude', 'int_magnitude', 'magtype', '0_10', '10_20', '20_30', '30_40', '40_50', '50_60', '60_70', '70_80', '80_90', '90_100']]

results_10_and_catalog.to_csv(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/all_results_10_all.csv', index = False)

results_10_and_catalog_filtered = results_10_and_catalog[~((results_10_and_catalog['dataset'] == 'sigloch') & (results_10_and_catalog['magnitude'] <= 7.2))]
# if dataset == 'sigloch':
# 	results_10_and_catalog_filtered = results_10_and_catalog[results_10_and_catalog['magnitude'] <= 7.2].copy()
# else:
# 	results_10_and_catalog_filtered = results_10_and_catalog.copy()

results_10_and_catalog_filtered.to_csv(f'/home/earthquakes1/homes/Rebecca/phd/stf/data/all_results_10.csv', index = False)



In [15]:
results_10_and_catalog

,event,dataset,catalog_id,year,month,day,hour,minute,latitude,longitude,...,0_10,10_20,20_30,30_40,40_50,50_60,60_70,70_80,80_90,90_100
0,19900405_2112,usgs,2811892,1990,04,05,21,12,15.1421,147.592900,...,0.283784,0.121622,0.121622,0.081081,0.040541,0.054054,0.040541,0.054054,0.067568,0.135135
1,19900418_1339,usgs,2812530,1990,04,18,13,39,1.2189,122.915600,...,0.240506,0.056962,0.063291,0.069620,0.069620,0.063291,0.063291,0.063291,0.063291,0.246835
2,19900530_0234,sigloch,2814540,1990,05,30,02,34,-6.0307,-77.214200,...,0.258621,0.258621,0.068966,0.034483,0.017241,0.034483,0.017241,0.051724,0.120690,0.137931
3,19901230_1914,usgs,2824394,1990,12,30,19,14,-5.1028,150.980300,...,0.321429,0.071429,0.047619,0.071429,0.083333,0.095238,0.059524,0.071429,0.059524,0.119048
4,19910422_2156,usgs,2829490,1991,04,22,21,56,9.6963,-83.074500,...,0.164502,0.077922,0.047619,0.038961,0.038961,0.060606,0.103896,0.108225,0.155844,0.203463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11646,20240817_1910,usgs,11875044,2024,08,17,19,10,52.9369,160.115800,...,0.263364,0.060626,0.049544,0.046284,0.046284,0.051499,0.057366,0.083442,0.122555,0.219035
11647,20241110_1649,usgs,11901608,2024,11,10,16,49,19.8191,-77.075700,...,0.513699,0.047089,0.032534,0.031678,0.029966,0.035103,0.041096,0.053082,0.071062,0.144692
11648,20241205_1844,usgs,11909383,2024,12,05,18,44,40.3740,-125.021667,...,0.281170,0.060433,0.054071,0.060433,0.083333,0.082061,0.053435,0.068702,0.087786,0.168575
11649,20241217_0147,usgs,11914214,2024,12,17,01,47,-17.6914,168.084200,...,0.431732,0.138825,0.067887,0.047292,0.037376,0.032799,0.033562,0.035850,0.048055,0.126621


In [16]:
results_10_and_catalog_filtered

,event,dataset,catalog_id,year,month,day,hour,minute,latitude,longitude,...,0_10,10_20,20_30,30_40,40_50,50_60,60_70,70_80,80_90,90_100
0,19900405_2112,usgs,2811892,1990,04,05,21,12,15.1421,147.592900,...,0.283784,0.121622,0.121622,0.081081,0.040541,0.054054,0.040541,0.054054,0.067568,0.135135
1,19900418_1339,usgs,2812530,1990,04,18,13,39,1.2189,122.915600,...,0.240506,0.056962,0.063291,0.069620,0.069620,0.063291,0.063291,0.063291,0.063291,0.246835
3,19901230_1914,usgs,2824394,1990,12,30,19,14,-5.1028,150.980300,...,0.321429,0.071429,0.047619,0.071429,0.083333,0.095238,0.059524,0.071429,0.059524,0.119048
4,19910422_2156,usgs,2829490,1991,04,22,21,56,9.6963,-83.074500,...,0.164502,0.077922,0.047619,0.038961,0.038961,0.060606,0.103896,0.108225,0.155844,0.203463
5,19910620_0518,usgs,2832224,1991,06,20,05,18,1.1920,122.821800,...,0.187500,0.068750,0.043750,0.050000,0.037500,0.056250,0.075000,0.062500,0.125000,0.293750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11646,20240817_1910,usgs,11875044,2024,08,17,19,10,52.9369,160.115800,...,0.263364,0.060626,0.049544,0.046284,0.046284,0.051499,0.057366,0.083442,0.122555,0.219035
11647,20241110_1649,usgs,11901608,2024,11,10,16,49,19.8191,-77.075700,...,0.513699,0.047089,0.032534,0.031678,0.029966,0.035103,0.041096,0.053082,0.071062,0.144692
11648,20241205_1844,usgs,11909383,2024,12,05,18,44,40.3740,-125.021667,...,0.281170,0.060433,0.054071,0.060433,0.083333,0.082061,0.053435,0.068702,0.087786,0.168575
11649,20241217_0147,usgs,11914214,2024,12,17,01,47,-17.6914,168.084200,...,0.431732,0.138825,0.067887,0.047292,0.037376,0.032799,0.033562,0.035850,0.048055,0.126621
